## Dependencies

In [ ]:
# Confirmed needed dependencies
import zillow
import random
import pprint

# Seth 

# Research how to make this truly invisible on GitHub Repo
# from config import zwsid
# Paste in your zwsid in the meantime using the following line
# zwsid = ""

In [15]:
# Possibly needed, but not yet used dependencies
import pandas as pd
import requests
import json

In [ ]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [ ]:
# Troy

# Established latitude range (30.33.00 to 30.09.00)
# Established longitude range (97.59.59 to 97.34.00)

#Use center point and radius to generate coordinates

# Generate random list of coordinates


## *TODO: Find a way to convert coordinates to residential addresses

In [ ]:
# Yuta

#Play with found sources to see if they will work

# Possible sources
# https://stackoverflow.com/questions/11390392/return-individual-address-components-city-state-etc-from-geopy-geocoder

# https://wiki.openstreetmap.org/wiki/Nominatim

In [ ]:
# Kat's block

## Zillow API Calls using Address and Zipcode

In [22]:
address = "929 Cavalier Ln" 
zipcode = "78757"

pp = pprint.PrettyPrinter(indent=4)

api = zillow.ValuationApi()


#This appears to be the ideal search to use with the most flushed out info
deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
pp.pprint(deep_results.get_dict())


# This appears to return the same as .GetZEstimate(),
# but has less accurate "extended_data" than .GetDeepSearchResults
# data = api.GetSearchResults(zwsid, address, zipcode)
# pp.pprint(data.get_dict())


# This appears to return the same as .GetSearchResults()
# detail_data = api.GetZEstimate(zwsid, data.zpid)
# pp.pprint(detail_data.get_dict())

{   'extended_data': {   'bathrooms': None,
                         'bedrooms': None,
                         'complete': False,
                         'finished_sqft': None,
                         'fips_county': None,
                         'last_sold_date': None,
                         'last_sold_price': None,
                         'lot_size_sqft': None,
                         'tax_assessment': None,
                         'tax_assessment_year': None,
                         'usecode': None,
                         'year_built': None},
    'full_address': {   'city': 'AUSTIN',
                        'latitude': '30.342641',
                        'longitude': '-97.718579',
                        'state': 'TX',
                        'street': '929 Cavalier Ln',
                        'zipcode': '78757'},
    'links': {   'comparables': 'http://www.zillow.com/homes/comps/119621219_zpid/',
                 'graphs_and_data': 'http://www.zillow.com/homedetails/92

## Calculate Value per Sqft

In [ ]:
# Seth

#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]
